In [3]:
pip install catboost

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.8/101.7 MB 1.5 MB/s eta 0:01:07
   ---------------------------------------- 1.0/101.7 MB 1.5 MB/s eta 0:01:06
    --------------------------------------- 1.3/101.7 MB 1.5 MB/s eta 0:01:06
    --------------------------------------- 1.6/101.7 MB 1.4 MB/s eta 0:01:10
    --------------------------------------- 1.8/101.7 MB 1.4 MB/s eta 0:01:10
    --------------------------------------- 2.1/101.7 MB 1.4 MB/s eta 0:01:11
    --------------------------------------- 2.4/101.7 MB 1.4 MB/s eta 0:01:10
   - -------------------------------------- 2.6/101.7 MB 1.4 MB/s eta 0:01:10
   - -------------------------------------- 2.9/101.7 MB 1.4 MB/s eta 0:01:11
   - -------------------------------------- 3.1/101.7 MB 1.3 MB/s eta 0:01:18
   - --

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [7]:
train_dir = 'C:/Users/Aziz/Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/train'  
test_dir = 'C:/Users/Aziz/Desktop/Alzheimer_s Dataset/Alzheimer_s Dataset/test'

In [9]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [11]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [13]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [15]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [17]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\Aziz\Downloads\Programs\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\Aziz\Downloads\Programs\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step


In [21]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step


In [23]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3251613	total: 186ms	remaining: 3m 5s
1:	learn: 1.2725736	total: 207ms	remaining: 1m 43s
2:	learn: 1.2293720	total: 226ms	remaining: 1m 15s
3:	learn: 1.1904939	total: 251ms	remaining: 1m 2s
4:	learn: 1.1587542	total: 271ms	remaining: 53.8s
5:	learn: 1.1300992	total: 290ms	remaining: 48.1s
6:	learn: 1.1048036	total: 309ms	remaining: 43.9s
7:	learn: 1.0817790	total: 330ms	remaining: 40.9s
8:	learn: 1.0603350	total: 353ms	remaining: 38.9s
9:	learn: 1.0408388	total: 372ms	remaining: 36.8s
10:	learn: 1.0250971	total: 391ms	remaining: 35.2s
11:	learn: 1.0116000	total: 410ms	remaining: 33.8s
12:	learn: 0.9975636	total: 430ms	remaining: 32.7s
13:	learn: 0.9837907	total: 452ms	remaining: 31.8s
14:	learn: 0.9732209	total: 470ms	remaining: 30.9s
15:	learn: 0.9627555	total: 488ms	remaining: 30s
16:	learn: 0.9535029	total: 505ms	remaining: 29.2s
17:	learn: 0.9445515	total: 523ms	remaining: 28.5s
18:	learn: 0.9352121	total: 542ms	remaining: 28s
19:	learn: 0.9273660	total: 559ms	remaining

In [24]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [27]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 56.14%


In [29]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [31]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [ ]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'C:\Users\m\Desktop\Alzheimer_s Dataset\test\NonDemented\27 (57).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")
